In [ ]:
!pip install beautifulsoup4 requests

In [8]:
from bs4 import BeautifulSoup
import requests

def get_naver_news_article_body(url):
    try:
        # User-Agent를 설정하여 봇 접근이 아님을 명시
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.raise_for_status() # HTTP 오류 발생 시 예외 발생
        response.encoding = 'utf-8' # 네이버는 주로 UTF-8 인코딩을 사용

        soup = BeautifulSoup(response.text, 'lxml')

        # 1. 기사 제목 추출 (선택 사항)
        # title = soup.find('h2', id='title_area')
        # article_title = title.text.strip() if title else "제목 없음"

        # 2. 기사 본문이 포함된 핵심 div 찾기
        # id="dic_area"를 가진 div를 먼저 찾아보는 것이 일반적
        article_body_div = soup.find('div', id='dic_area')

        if not article_body_div:
            # id="dic_area"가 없는 경우, 다른 본문 관련 클래스를 시도 (예: _article_body)
            article_body_div = soup.find('div', class_='_article_body')

        if not article_body_div:
            # 그래도 못 찾으면, 다른 가능성 있는 본문 영역 ID/클래스를 추가할 수 있음
            # 예: article_body_div = soup.find('div', class_='news_contents')
            print(f"[{url}] 본문 영역 div를 찾을 수 없습니다.")
            return None, None # 제목, 본문 모두 None 반환

        # 3. 본문 내 불필요한 태그 제거 (스크립트, 광고, 이미지 캡션 등)
        # 이미지를 텍스트 본문에서 제거하고 싶을 때 유용
        unwanted_elements = [
            'script', 'style', 'a', # 스크립트, 스타일, 링크 태그 제거
            'span', # 불필요한 span 태그 제거 (예: 기자 이름, 날짜 등)
            'strong', # 볼드 태그는 텍스트로 보존하되, 특정 경우가 아니면 제거를 고려
            'div.byline', # 기자 정보 (가끔 본문 div 안에 있음)
            'div.util_art', # 본문 하단 공유 버튼 등 유틸리티
            'div.text_guide', # (종종 보이는) 기사 중간의 "AI 요약 본문입니다" 같은 문구
            'div.img_center', 'div.img_viewer', 'figure', # 이미지/사진 관련 div 제거
            'em.img_desc', 'figcaption' # 이미지 캡션 제거
        ]

        for selector in unwanted_elements:
            for match in article_body_div.select(selector):
                match.decompose()

        # 4. <p> 태그만 추출하여 본문 텍스트 생성
        paragraphs = article_body_div.find_all('p')
        if not paragraphs: # p 태그가 없는 경우 (다른 태그로 본문이 구성된 경우)
             # Fallback: p 태그가 없으면 전체 텍스트를 가져오되, 추가 정제 필요
            main_text = article_body_div.get_text(separator='\n', strip=True)
            print(f"[{url}] <p> 태그를 찾을 수 없어 본문 div 전체 텍스트를 추출합니다.")
        else:
            cleaned_paragraphs = []
            for p_tag in paragraphs:
                # p 태그 내에 불필요한 내용(예: (사진=...) )이 있다면 추가로 제거
                # 이 부분은 필요에 따라 정규식을 사용할 수도 있습니다.
                paragraph_text = p_tag.get_text(strip=True)
                if paragraph_text: # 빈 단락은 제외
                    cleaned_paragraphs.append(paragraph_text)

            main_text = '\n\n'.join(cleaned_paragraphs) # 각 단락 사이에 두 줄 공백
            # 추가적인 공백이나 특수문자 정제
            main_text = main_text.replace('\n\n\n', '\n\n').replace('  ', ' ').strip()


        # 기사 제목도 여기서 추출
        title_tag = soup.find('h2', id='title_area')
        article_title = title_tag.text.strip() if title_tag else "제목 없음"

        return article_title, main_text

    except requests.exceptions.RequestException as e:
        print(f"URL 접근 중 오류 발생: {e}")
        return None, None
    except Exception as e:
        print(f"본문 추출 중 알 수 없는 오류 발생: {e}")
        return None, None

# --- 사용 예시 ---
# 실제 네이버 뉴스 기사 URL (최신 기사로 업데이트)
naver_news_url = "https://n.news.naver.com/mnews/article/003/0013368152"

title, body = get_naver_news_article_body(naver_news_url)

if title and body:
    print(f"\n--- 기사 제목 ---\n{title}\n")
    print(f"--- 기사 본문 ({len(body)}자) ---\n{body}")
else:
    print("기사 본문을 추출하지 못했습니다.")

[https://n.news.naver.com/mnews/article/003/0013368152] <p> 태그를 찾을 수 없어 본문 div 전체 텍스트를 추출합니다.

--- 기사 제목 ---
'o1 개발자' 오픈AI 한국인 연구원도 저커버그 품으로?

--- 기사 본문 (1126자) ---
[서울=뉴시스]윤정민 기자 = 오픈AI 인공지능(AI) 추론 특화 멀티모달 모델 'o1' 개발자로 알려진 한국인 오픈AI 개발자가 메타로 이직할 예정인 것으로 전해졌다.
16일(현지 시간) 와이어드 등에 따르면 정형원 오픈AI 연구원은 제이슨 웨이 연구원과 함께 메타 초지능연구소(MSL)에 합류할 예정이다.
미국 매사추세츠공대(MIT) 박사 출신인 정 연구원은 2019년 구글 브레인(현 구글 리서치 산하 AI 연구팀, 현 구글 딥마인드)에서 거대언어모델(LLM) 등을 연구한 후 2023년 2월 오픈AI로 이직해 추론 모델, 에이전트 개발에 활동했다.
대표적으로 'o1' 모델과 검색과 추론 등을 걸쳐 보고서를 작성하는 AI 에이전트 '딥 리서치' 등 개발에 참여했다. 특히 정 연구원은 오픈AI 공식 유튜브에 출연해 o1 특징 중 하나로 "윈공진능 헷꺌리께 만둘 쑤 있닳(인공지능 헷갈리게 만들 수 있다)"는 등의 한국인도 이해하기 어려울 수 있는 한국어 문장을 영어로 번역하도록 해 화제를 모았다.
특히 최근에는 오픈AI가 행사 생방송을 진행하는 과정에서 샘 올트먼 오픈AI 최고경영자(CEO)와 함께 붙어있는 모습을 보이기도 했다. 하지만 정 연구원 오픈AI 슬랙 프로필이 현재 비활성화됐다며 이미 퇴사한 것 같다는 보도가 이어졌다.
정 연구원 이동에는 웨이 연구원 퇴사와 맞물린 것 같다는 해석이 나왔다. 웨이 연구원 역시 'o3' 모델 등 개발에 참여했는데 두 연구원은 구글에서도 함께 일한 바 있다.
하지만 정 연구원 링크드인에는 오픈AI 재직 중으로 나타나 있다. 오픈AI 측은 아직 정 연구원 퇴사 여부에 대한 입장을 밝히지 않았다.
한편 두 연구원의 메타 합류설이 나온 데